# Fetch Data from Person category from COCO

1. Run the get_coco.sh file in yolov5/data/scripts/  to get the YOLO Dataset and annotations 

In [47]:
from pycocotools.coco import COCO
import requests
import os
from shutil import copy

coco = COCO('/home/pappacha/softwares/yolov5/data/coco/annotations/instances_val2017.json') # path to the json file of annotations
cats = coco.loadCats(coco.getCatIds()) #loads the category
nms=[cat['name'] for cat in cats] #select the names in category
print('COCO categories: \n{}\n'.format(' '.join(nms)))

catIds = coco.getCatIds(catNms=['person'])  # selects particularly person
imgIds = coco.getImgIds(catIds=catIds ) #Fetches Category ID
images = coco.loadImgs(imgIds) #Fetches Image ID 

print("images: ", len(images)) # Total number of images in  Person Category


#To fetch the labels of the images under person category
for im in images:
    file_name = os.path.splitext(os.path.basename(im['file_name']))[0] #fetch the file name without extension
    base_source = '/home/pappacha/softwares/yolov5/data/coco/'  #path where the images are already downloaded by following the first step  ( get_coco.sh)
    base_dest = '/home/pappacha/softwares/yolov5/total_data/persons' #path where the images wants to be stored for training 
    copy(os.path.join(base_source,'images','val2017',os.path.basename(im['file_name'])), os.path.join(base_dest,'images',os.path.basename(im['file_name']))) #Copy the image from source to destination 
    txt_file = file_name + '.txt' # Since the labels and Image have the same file name 
    copy(os.path.join(base_source,'labels','val2017',txt_file), os.path.join(base_dest,'labels',txt_file)) #Copy the txt from source to destination

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

images:  2693


# If you want to download annotations particulary belonging to persons in a csv file  

In [32]:
#Download annotations
import csv
classes = 'person'
with open('annotations_download_' + classes + '.csv', mode='w', newline='') as annot:
    for im in images:
        annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        #print(anns)
        #print(len(anns))
        for i in range(len(anns)):
            print(anns[i])
            annot_writer = csv.writer(annot)
            #annot_writer.writerow([im['coco_url'], anns[i]['bbox'][0], anns[i]['bbox'][1], anns[i]['bbox'][0] + anns[i]['bbox'][2], anns[i]['bbox'][1] + anns[i]['bbox'][3], classes])
            annot_writer.writerow(['downloaded_images' + im['file_name'], int(round(anns[i]['bbox'][0])), int(round(anns[i]['bbox'][1])), int(round(anns[i]['bbox'][0] + anns[i]['bbox'][2])), int(round(anns[i]['bbox'][1] + anns[i]['bbox'][3])), classes])
            #print("anns: ", im['coco_url'], anns[i]['bbox'][0], anns[i]['bbox'][1], anns[i]['bbox'][0] + anns[i]['bbox'][2], anns[i]['bbox'][1] + anns[i]['bbox'][3], 'person')
    annot.close()

2
{'segmentation': [[253.85, 187.23, 250.82, 193.01, 255.22, 199.6, 260.99, 202.35, 270.89, 204.55, 273.64, 211.15, 277.21, 222.15, 284.09, 228.47, 293.71, 232.04, 303.06, 233.14, 311.03, 231.77, 317.9, 224.9, 320.93, 215.55, 320.1, 206.48, 315.43, 199.05, 310.2, 193.56, 308.83, 186.13, 299.48, 182.28, 294.81, 182.01, 288.49, 175.96, 284.09, 169.36, 273.64, 168.26, 273.09, 172.94, 277.49, 177.34, 283.81, 181.46, 287.94, 184.48, 283.26, 188.06, 281.61, 190.53, 283.54, 196.58, 284.36, 202.9, 279.96, 198.5, 272.54, 194.93, 266.77, 195.21, 265.12, 194.38, 259.89, 189.43, 254.12, 186.96]], 'area': 2188.0864999999994, 'iscrowd': 0, 'image_id': 532481, 'bbox': [250.82, 168.26, 70.11, 64.88], 'category_id': 1, 'id': 508910}
{'segmentation': [[295.86, 372.43, 293.99, 376.41, 293.99, 376.41, 297.62, 377.35, 296.8, 389.18, 296.8, 389.3, 286.49, 389.77, 289.07, 386.13, 287.19, 385.08, 286.73, 380.28, 285.55, 378.87, 287.31, 377.23, 289.07, 376.06, 291.06, 375.48, 290.82, 372.55, 292.0, 370.68, 293

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# If you want to keep the labels only from one particular category in a folder

In [1]:
import os
path = '/home/pappacha/softwares/yolov5/ETAI_YOLO/total_data/COCOUPDATED/labels/'
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and '00000' in i: # Incase you have a folder with different types of images .. in mu case operators and coco and I wanted to particularly change only the coco . Hence the "and" condition 
        links_to_keep = []
        with open(os.path.join(path,i), "r") as f:
            for line in f.readlines():
                if line.startswith("0"): #checks if the line starts with 0
                    links_to_keep.append(line)

        with open(os.path.join(path,i), "w") as f: #writes the data in the file
            for link in links_to_keep:
                f.write(link)

# If you want to check that the file has only person category and save the file name in a list 

In [49]:
import os
path = '/home/pappacha/softwares/yolov5/ETAI_YOLO/total_data/persons/labels/'
fname_to_keep = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and '00000' in i: # Incase you have a folder with different types of images .. in mu case operators and coco and I wanted to particularly change only the coco . Hence the "and" condition 
        
        with open(os.path.join(path,i), "r") as f:
            for line in f.readlines():
                if not line.startswith("0"): #checks if the line not starts with 0
                    #links_to_keep.append(line)
                    if i in fname_to_keep:
                        fname_to_keep.remove(i)
                        
                    break
                else:
                    if i in fname_to_keep: #checks if the filename is present in the list
                        pass
                    else:
                        fname_to_keep.append(i)
                        #print(fname_to_keep)
                        
                    
                

In [50]:
fname_to_keep

['000000439522.txt',
 '000000252559.txt',
 '000000329323.txt',
 '000000121497.txt',
 '000000398905.txt',
 '000000212166.txt',
 '000000409475.txt',
 '000000144784.txt',
 '000000018519.txt',
 '000000259854.txt',
 '000000049259.txt',
 '000000066886.txt',
 '000000354307.txt',
 '000000448810.txt',
 '000000488710.txt',
 '000000130599.txt',
 '000000263425.txt',
 '000000202228.txt']

# Check if the Files have other classes

In [3]:
import os
path = '/home/pappacha/softwares/yolov5/ETAI_YOLO/total_data/persons/labels/'
fname_to_keep = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and '00000' in i: # Incase you have a folder with different types of images .. in mu case operators and coco and I wanted to particularly change only the coco . Hence the "and" condition 
        
        with open(os.path.join(path,i), "r") as f:
            for line in f.readlines():
                if not line.startswith("0"): #checks if the line not starts with 0
                    #links_to_keep.append(line)
                    if i in fname_to_keep:
                        fname_to_keep.append(i)
                        